In [253]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import timedelta

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/paid-user-visits/startup_users_visits.csv
/kaggle/input/paid-user-visits/test.csv


In [254]:
df = pd.read_csv('/kaggle/input/paid-user-visits/startup_users_visits.csv')

In [255]:
df.head(10)

,user_id,date,pay
0,0001599900972add,2023-06-15,False
1,0001599900972add,2023-06-16,False
2,00019b04f17ade68,2022-12-18,False
3,0001c405d5a77893,2023-07-09,False
4,000314ab1fe38c66,2023-05-12,False
5,000314ab1fe38c66,2023-05-29,False
6,000314ab1fe38c66,2023-05-28,False
7,000314ab1fe38c66,2023-05-27,False
8,000314ab1fe38c66,2023-05-26,False
9,000314ab1fe38c66,2023-05-25,False


In [256]:
df.describe()

,user_id,date,pay
count,373366,373366,373366
unique,25648,317,2
top,38c773dff3e5f7c4,2023-09-28,False
freq,255,1864,365898


In [257]:
df['date'] = pd.to_datetime(df['date'])

In [258]:
last_update = df['date'].max()

In [259]:
df = df.sort_values(by=['user_id', 'date'])

In [260]:
df['first_use_date'] = df.groupby('user_id')['date'].transform('first')

In [261]:
df['days_from_start'] = (df['date'] - df['first_use_date']).dt.days

In [262]:
df = df[(df['days_from_start'] >= 0) & (df['days_from_start'] <= 89)]

In [263]:
paid_users = df[df['pay']].drop_duplicates(subset='user_id')

In [264]:
df = df[df['pay']].drop_duplicates(subset=['user_id'])

In [265]:
result = paid_users.groupby(paid_users['first_use_date'].dt.to_period('M'))['user_id'].nunique().reset_index()

In [266]:
result['first_use_date'] = result['first_use_date'].dt.to_timestamp()

In [267]:
result = result[((last_update - result['first_use_date']).dt.days) >= 90]

In [268]:
result.head(10)

,first_use_date,user_id
0,2022-11-01,133
1,2022-12-01,552
2,2023-01-01,437
3,2023-02-01,949
4,2023-03-01,950
5,2023-04-01,678
6,2023-05-01,954
7,2023-06-01,848
8,2023-07-01,698


In [269]:
result.to_csv('submission.csv', index=False)